# Coursera Capstone Project: The Battle of Neighbourhoods 

### Importing required libraries and packages

In [42]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Creating variables for Foursquare

In [43]:
CLIENT_ID = '4PT3EL2LA4VMKA2GUMND4ED5RMGLYS3D0SBTTL1UQ5IA2GC3' # your Foursquare ID
CLIENT_SECRET = 'YourFoursquareSecret' # your Foursquare Secret
VERSION = '20200612' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 4PT3EL2LA4VMKA2GUMND4ED5RMGLYS3D0SBTTL1UQ5IA2GC3


### Creating URL for category: Gym/ Fitness centres and 
###                  5 cities: New York, Chicago, San Francisco, Jersey City and Boston

In [44]:
# 4bf58dd8d48988d175941735 = categoryId for GYM/ Fitness centres
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
     "4bf58dd8d48988d175941735" ) # GYM/Fitness Centre CATEGORY ID
    results[city] = requests.get(url).json()

In [45]:
results[city]

{'meta': {'code': 200, 'requestId': '5ee39e4dcb8236079154b755'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'boston ma',
   'center': {'lat': 42.35843, 'lng': -71.05977},
   'displayString': 'Boston, MA, United States',
   'cc': 'US',
   'geometry': {'bounds': {'ne': {'lat': 42.400819999999996,
      'lng': -70.748802},
     'sw': {'lat': 42.22788, 'lng': -71.191155}}},
   'slug': 'boston-massachusetts',
   'longId': '72057594042858892'},
  'headerLocation': 'Boston',
  'headerFullLocation': 'Boston',
  'headerLocationGranularity': 'city',
  'query': 'gym fitness',
  'totalResults': 190,
  'suggestedBounds': {'ne': {'lat': 42.39359533038312,
    'lng': -71.00810942916115},
   'sw': {'lat': 42.221665763076075, 'lng': -71.17683766882203}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 

### Getting venues for selected cities and category

In [46]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

 The Foursquare API Only gives us the nearest 100 venues in the city.

Let us first check out their densities.

In [47]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='yellow',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Gym/Fitness Center in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Gym/Fitness Center in New York, NY =  261
Showing Top 100
Total number of Gym/Fitness Center in Chicago, IL =  240
Showing Top 100
Total number of Gym/Fitness Center in San Francisco, CA =  242
Showing Top 100
Total number of Gym/Fitness Center in Jersey City, NJ =  154
Showing Top 100
Total number of Gym/Fitness Center in Boston, MA =  190
Showing Top 100


#### Map for City1: NEW YORK

In [48]:
# New York
maps[cities[0]]


#### Map for City2: CHICAGO

In [49]:
#Chicago
maps[cities[1]]

#### Map for City3: SAN FRANCISCO 

In [50]:
#San Fransisco
maps[cities[2]]

#### Map for City4: JERSEY CITY

In [51]:
#New Jersey
maps[cities[3]]

#### Map for City4: BOSTON

In [52]:
#Boston
maps[cities[4]]

Let  us have a concrete measure of this density.



For this we will use some basic statistics.

We will get the mean location of the  Gym/Fitness Centres.

#### Next we will take the average of the distance of the venues to the mean coordinates.

In [53]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='yellow',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02592323858922523
Chicago, IL
Mean Distance from Mean coordinates
0.0461274800911952
San Francisco, CA
Mean Distance from Mean coordinates
0.02555210180662777
Jersey City, NJ
Mean Distance from Mean coordinates
0.012396274548575677
Boston, MA
Mean Distance from Mean coordinates
0.03664967004230272


### Visualizing the Mean Distance from Mean coordinates on the maps for respective cities

In [56]:
# City: New York
maps[cities[0]]

In [57]:
# City: Chicago
maps[cities[1]]

In [58]:
# City: San Francisco
maps[cities[2]]

In [59]:
# City: Jersey City
maps[cities[3]]

In [60]:
# City: Boston
maps[cities[4]]

The mean distance is the highest for Chicago, indicating that gyms are sparsely located in the city.
Hence, Chicago is the best option for setting up a gym or fitness place. 

In [61]:
# Re-calculating the Mean Distance from Mean coordinates for Chicago
city = 'Chicago, IL'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# CONSIDER THE BIGGEST DISTANCE

Chicago, IL
Mean Distance from Mean coordinates
0.044256723456627796


### Happy Fitness Day!!
And By that I conclude my Notebook